In [ ]:
%matplotlib inline


# AFQ API

An example using the AFQ API


In [ ]:
import os.path as op

import matplotlib.pyplot as plt
import nibabel as nib
import plotly

from AFQ.api.group import GroupAFQ
import AFQ.data as afd

## Get some example data

Retrieves High angular resolution diffusion imaging (HARDI) dataset from
Stanford's Vista Lab

  see https://purl.stanford.edu/ng782rw8378 for details on dataset.

The data for the first subject and first session are downloaded locally
(by default into the users home directory) under:

  ``.dipy/stanford_hardi/``

Anatomical data (``anat``) and Diffusion-weighted imaging data (``dwi``) are
then extracted, formatted to be BIDS compliant, and placed in the AFQ
data directory (by default in the users home directory) under:

  ``AFQ_data/stanford_hardi/``

This data represents the required preprocessed diffusion data necessary for
intializing the AFQ object (which we will do next)

The clear_previous_afq is used to remove any previous runs of the afq object
stored in the AFQ_data/stanford_hardi/ BIDS directory. Set it to false if
you want to use the results of previous runs.



In [ ]:
afd.organize_stanford_data(clear_previous_afq=True)

## Initialize an AFQ object:

Creates an AFQ object, that encapsulates tractometry. This object can be
used to manage the entire AFQ pipeline, including:

- Tractography
- Registration
- Segmentation
- Cleaning
- Profiling
- Visualization

In this example we will load the subjects session data from the previous step
using the default AFQ parameters.

<div class="alert alert-info"><h4>Note</h4><p>The first time intializing the AFQ object will download necessary
   waypoint regions of interest (ROIs) templates into AFQ data directory:</p></div>

- Human corpus callosum templates: ``AFQ_data/callosum_templates/``

  see https://digital.lib.washington.edu/researchworks/handle/1773/34926

- Tract probability maps: ``AFQ_data/templates/``

  see https://figshare.com/articles/Tract_probability_maps_for_automated_fiber_quantification/6270434  # noqa

These waypoints ROIs will used to identify the desired white matter tracts.

This will also create an output folder for the corresponding AFQ derivatives
in the AFQ data directory: ``AFQ_data/stanford_hardi/derivatives/afq/``

To initialize this object we will pass in the path location to our BIDS
compliant data.

<div class="alert alert-info"><h4>Note</h4><p>As noted above, the Stanford HARDI data contains anatomical and
   diffusion weighted imaging (dwi) data. In this example, we are interested
   in the vistasoft dwi. For our dataset the `dmriprep` is optional, but
   we have included it to make the initialization more explicit.</p></div>

<div class="alert alert-info"><h4>Note</h4><p>We will also be using plotly to generate an interactive visualization.
   So we will specify plotly_no_gif as the visualization backend.</p></div>



In [ ]:
myafq = GroupAFQ(
    bids_path=op.join(afd.afq_home, 'stanford_hardi'),
    preproc_pipeline='vistasoft',
    viz_backend_spec='plotly_no_gif')

## Reading in DTI FA (Diffusion Tensor Imaging Fractional Anisotropy)
The AFQ object holds a table with file names to various data derivatives.

For example, the file where the FA computed from DTI is stored can be
retrieved by inspecting the ``dti_fa`` property. The measures are stored
in a series, and since we only have one subject and one session we will
access the first (and only) file name from the example data.

<div class="alert alert-info"><h4>Note</h4><p>The AFQ API computes quantities lazily. This means that DTI parameters
   are not computed until they are required. This means that the first
   line below is the one that requires time.</p></div>

We will then use `nibabel` to load the deriviative file and retrieve the
data array.



In [ ]:
FA_fname = myafq.dti_fa["01"]
FA_img = nib.load(FA_fname)
FA = FA_img.get_fdata()

## Visualize the result with Matplotlib
At this point `FA` is an array, and we can use standard Python tools to
visualize it or perform additional computations with it.

In this case we are going to take an axial slice halfway through the
FA data array and plot using a sequential color map.

<div class="alert alert-info"><h4>Note</h4><p>The data array is structured as a xyz coordinate system.</p></div>



In [ ]:
fig, ax = plt.subplots(1)
ax.matshow(FA[:, :, FA.shape[-1] // 2], cmap='viridis')
ax.axis("off")

## Visualizing bundles and tract profiles:
The pyAFQ API provides several ways to visualize bundles and profiles.

First, we will run a function that exports an html file that contains
an interactive visualization of the bundles that are segmented.

<div class="alert alert-info"><h4>Note</h4><p>By default we resample a 100 points within a bundle, however to reduce
   processing time we will only resample 50 points.</p></div>

Once it is done running, it should pop a browser window open and let you
interact with the bundles.

<div class="alert alert-info"><h4>Note</h4><p>Running the code below triggers the full pipeline of operations
   leading to the computation of the tract profiles. Therefore, it
   takes a little while to run (about 40 minutes, typically).</p></div>

<div class="alert alert-info"><h4>Note</h4><p>You can hide or show a bundle by clicking the legend, or select a
   single bundle by double clicking the legend. The interactive
   visualization will also all you to pan, zoom, and rotate.</p></div>



In [ ]:
bundle_html = myafq.all_bundles_figure
plotly.io.show(bundle_html["01"])

We can also visualize the tract profiles in all of the bundles. These
plots show both FA (left) and MD (right) layed out anatomically.
To make this plots, it is required that you install with
`pip install pyAFQ[plot]` so that you have the necessary dependencies.




In [ ]:
fig_files = myafq.tract_profile_plots["01"]

.. figure:: {{ fig_files[0] }}


